- SPSA optimizer, optimization loop(to learn the DD parameters)
- cost fuction (1-fidelity function) with parameters
- Integrate the optimizer with transpiler pass(cooperate with others)
- baseline comparisons(naive vs. LDD)



In [2]:
import numpy as np
import qiskit
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import PauliTwoDesign
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp


In [1]:
%pip show qiskit-algorithms
%pip show qiskit

Name: qiskit-algorithms
Version: 0.3.0
Summary: Qiskit Algorithms: A library of quantum computing algorithms
Home-page: https://github.com/qiskit-community/qiskit-algorithms
Author: Qiskit Algorithms Development Team
Author-email: qiskit@us.ibm.com
License: Apache-2.0
Location: c:\Users\yaxua\OneDrive\桌面\hackathon\.venv\Lib\site-packages
Requires: numpy, qiskit, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.
Name: qiskit
Version: 1.4.3
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: https://www.ibm.com/quantum/qiskit
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: c:\Users\yaxua\OneDrive\桌面\hackathon\.venv\Lib\site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-algorithms, qiskit-ibm-runtime
Note: you may need to restar

In [3]:

LDD_circuit = PauliTwoDesign(2, reps=1, seed=2) #the circuit with trainable parameters
cost_function = SparsePauliOp("ZZ") #cost function to minimize as much as possible
initial_point = np.random.random(LDD_circuit.num_parameters)
estimator = Estimator()

def loss(x):
    job = estimator.run([LDD_circuit], [cost_function], [x])
    return job.result().values[0]

spsa = SPSA(maxiter=300)
result = spsa.minimize(loss, x0=initial_point)
result.x

C:\Users\yaxua\AppData\Local\Temp\ipykernel_10644\293113102.py:4: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  estimator = Estimator()


array([-1.77422394,  2.74465009, -2.21485825,  3.42563519])

In [5]:
print(f"before training: loss={estimator.run([LDD_circuit], [cost_function], [initial_point]).result().values[0]}")
print(f"after training: loss={estimator.run([LDD_circuit], [cost_function], [result.x]).result().values[0]}")


before training: loss=0.5295914479196043
after training: loss=-0.8660203577504675


In [ ]:
# maybe later
class TerminationChecker:

    def __init__(self, J : int):
        self.J = J #cost function
        self.values = []

    def __call__(self, nfev, parameters, value, stepsize, accepted) -> bool:
        self.values.append(value)

        if len(self.values) > self.J:
            last_values = self.values[-self.J:]
            pp = np.polyfit(range(self.J), last_values, 1)
            slope = pp[0] / self.J

            if slope > 0:
                return True
        return False